In [0]:
%run ./01_Project_config

In [0]:

from pyspark.sql.functions import *

bronze_df = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("cloudFiles.inferColumnTypes", "true")
        
        .option("cloudFiles.schemaLocation", f"{paths['checkpoints']}/schema/card_txn")
        .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
        .option("cloudFiles.rescuedDataColumn", "_rescued_data")
        .option("header", "true")
       
        .load(paths["transactions"])
)


bronze_enriched = (
    bronze_df
        .withColumn("ingestion_timestamp", current_timestamp())
        .withColumn("source_file", col("_metadata.file_path"))
)

(
    bronze_enriched.writeStream
        .format("delta")
        .outputMode("append")
        .option("checkpointLocation", f"{paths['checkpoints']}/bronze_card_txn")
        .option("mergeSchema", "true")
        .trigger(availableNow=True)
        # Dynamic table naming based on config
        .toTable(f"{catalog}.{bronze_schema}.card_transactions")
)

In [0]:
%sql
SELECT source_file, COUNT(*)
FROM fraud_lakehouse.bronze.card_transactions
GROUP BY source_file;